In [6]:
API_KEY = "YiqUaoXU4B9HeUTZ0boaLNMkvTaeqwVDbz5CTu6vU1KNQLq0WewRdeehZaV4FPRT"
API_SECRET = "6IrfZdx8SdADJQLZRqrtEfCZrtkLTzCN8pt1Kyx9dymKG34Im9VvR3EceUZykyMk"

In [43]:
import ccxt

exchange_id = 'binance'
exchange_class = getattr(ccxt, exchange_id)
exchange = exchange_class({
    'apiKey': API_KEY,
    'secret': API_SECRET,
    'options': {
        'createMarketBuyOrderRequiresPrice': False, # switch off
    },
})

In [8]:
exchange.fetch_balance()

{'info': {'makerCommission': '10',
  'takerCommission': '10',
  'buyerCommission': '0',
  'sellerCommission': '0',
  'canTrade': True,
  'canWithdraw': True,
  'canDeposit': True,
  'updateTime': '1622576488259',
  'accountType': 'SPOT',
  'balances': [{'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'BNB', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'BNT', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'GAS', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'BCC', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset':

In [60]:
balances = exchange.fetch_balance()['info']['balances']
print(next((item for item in balances if item["asset"] == "USDT"), None))
print(next((item for item in balances if item["asset"] == "BTC"), None))


{'asset': 'USDT', 'free': '703.27782410', 'locked': '0.00000000'}
{'asset': 'BTC', 'free': '0.00218781', 'locked': '0.00000000'}


In [62]:
def get_balance(symbol):
    balances = exchange.fetch_balance()['info']['balances']
    return float(next((item["free"] for item in balances if item["asset"] == symbol), None))

print(get_balance("USDT"))
print(get_balance("BTC"))
print(get_balance("ETH"))

703.2778241
0.00218781
0.02569836


In [93]:
#exchange.load_markets()
#      time.sleep(exchange.rateLimit / 1000) # time.sleep wants seconds
exchange.load_markets()['DOGE/USDT']['limits']['amount']['min']

1.0

In [106]:
symbols = ['BTC/USDT', 'ETH/USDT', 'DOGE/USDT', 'LTC/USDT', 'MATIC/USDT']

for symbol in symbols:
    balance_crypto = get_balance(symbol.split('/')[0])
    min_size = exchange.load_markets()[symbol]['limits']['amount']['min']
    if balance_crypto > min_size:
        print("trade de venta pa: " + str(exchange.create_market_sell_order(symbol, balance_crypto)))




trade de venta pa: {'info': {'symbol': 'DOGEUSDT', 'orderId': '1765438229', 'orderListId': '-1', 'clientOrderId': 'x-R4BD3S82511a8905bee0c119ac74a7', 'transactTime': '1630443940132', 'price': '0.00000000', 'origQty': '487.00000000', 'executedQty': '487.00000000', 'cummulativeQuoteQty': '134.55810000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'fills': [{'price': '0.27630000', 'qty': '487.00000000', 'commission': '0.13455810', 'commissionAsset': 'USDT', 'tradeId': '325388162'}]}, 'id': '1765438229', 'clientOrderId': 'x-R4BD3S82511a8905bee0c119ac74a7', 'timestamp': 1630443940132, 'datetime': '2021-08-31T21:05:40.132Z', 'lastTradeTimestamp': None, 'symbol': 'DOGE/USDT', 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'side': 'sell', 'price': 0.2763, 'stopPrice': None, 'amount': 487.0, 'cost': 134.5581, 'average': 0.2763, 'filled': 487.0, 'remaining': 0.0, 'status': 'closed', 'fee': {'cost': 0.1345581, 'currency': 'USDT'}, 'trades': [{'info': {'

In [105]:
# Python
symbols = ['BTC/USDT', 'ETH/USDT', 'DOGE/USDT', 'LTC/USDT', 'MATIC/USDT']

window_short = 70
window_long = 130

candles = {}
long = {}
min_size = {}
open_order = {}

import time

balance_usd_order = get_balance(symbol.split('/')[1]) / (len(symbols)+1)
# print(f"{balance_usd:.2f} {symbol.split('/')[1]}")


while True:
    if exchange.has['fetchOHLCV']:
        starttime = time.time()
        # guardo las candles iniciales:
        for symbol in symbols:
            received_candles = exchange.fetch_ohlcv(symbol, '1m',params={
                    'limit':window_long})
            candles[symbol] = [received[4] for received in received_candles] # Me quedo con el precio de cierre
            
            min_size[symbol] = exchange.load_markets()[symbol]['limits']['amount']['min']
        
        for symbol in symbols:
            
            received = exchange.fetch_ohlcv(symbol, '1m',params={
                    'limit':1})[0]

            # Me guardo la ultima vela
            candles[symbol].append(received[4]) # Me quedo con el precio de cierre

            balance_crypto = get_balance(symbol.split('/')[0])
    
            if balance_crypto > min_size[symbol]:
                long[symbol] = True
                print(f"Available balance: {balance_crypto} {symbol.split('/')[0]}")
            else:
                long[symbol] = False

            # Calculo los MA
            short_ma = sum(candles[symbol][-window_short:])/len(candles[symbol][-window_short:])
            long_ma = sum(candles[symbol][-window_long:])/len(candles[symbol][-window_long:])

            print(f"{symbol.split('/')[0]}: Short ma: {short_ma:.2f}, Long ma: {long_ma:.2f}")
            candles[symbol] = candles[symbol][-window_long:]

            # Y ahora tradeo
            if not long[symbol]: # esto significa que tengo usdt
                if short_ma > long_ma:
                    # compro pa
                    print("trade de compra pa: " + str(exchange.create_market_buy_order(symbol,0, params={
                        'quoteOrderQty' : balance_usd_order,
                    })))
                    long[symbol] = True
            else: # esto significa que tengo crypto
                if short_ma < long_ma:
                    # Vendo pa
                    print("trade de venta pa: " + str(exchange.create_market_sell_order(symbol, balance_crypto)))
                    long[symbol]=False
        
        time.sleep(60.0 - ((time.time() - starttime) % 60.0)) 

BTC: Short ma: 47406.38, Long ma: 47368.93
trade de compra pa: {'info': {'symbol': 'BTCUSDT', 'orderId': '7372494030', 'orderListId': '-1', 'clientOrderId': 'x-R4BD3S827e0842294743beff9d8477', 'transactTime': '1630429579627', 'price': '0.00000000', 'origQty': '0.00286000', 'executedQty': '0.00286000', 'cummulativeQuoteQty': '135.42566180', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'fills': [{'price': '47351.63000000', 'qty': '0.00286000', 'commission': '0.00000286', 'commissionAsset': 'BTC', 'tradeId': '1032046937'}]}, 'id': '7372494030', 'clientOrderId': 'x-R4BD3S827e0842294743beff9d8477', 'timestamp': 1630429579627, 'datetime': '2021-08-31T17:06:19.627Z', 'lastTradeTimestamp': None, 'symbol': 'BTC/USDT', 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'side': 'buy', 'price': 47351.63, 'stopPrice': None, 'amount': 0.00286, 'cost': 135.4256618, 'average': 47351.63, 'filled': 0.00286, 'remaining': 0.0, 'status': 'closed', 'fee': {'cost': 2.86e

Available balance: 0.789118 LTC
LTC: Short ma: 172.35, Long ma: 171.50
MATIC: Short ma: 1.35, Long ma: 1.35
Available balance: 0.00286036 BTC
BTC: Short ma: 47405.54, Long ma: 47374.79
Available balance: 0.04012446 ETH
ETH: Short ma: 3391.57, Long ma: 3387.17
Available balance: 484.537 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
Available balance: 0.789118 LTC
LTC: Short ma: 172.39, Long ma: 171.54
MATIC: Short ma: 1.35, Long ma: 1.35
Available balance: 0.00286036 BTC
BTC: Short ma: 47404.41, Long ma: 47375.57
Available balance: 0.04012446 ETH
ETH: Short ma: 3391.25, Long ma: 3387.21
Available balance: 484.537 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
Available balance: 0.789118 LTC
LTC: Short ma: 172.41, Long ma: 171.56
MATIC: Short ma: 1.35, Long ma: 1.35
Available balance: 0.00286036 BTC
BTC: Short ma: 47403.86, Long ma: 47376.34
Available balance: 0.04012446 ETH
ETH: Short ma: 3390.88, Long ma: 3387.20
Available balance: 484.537 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
Available balanc

Available balance: 0.789118 LTC
LTC: Short ma: 172.70, Long ma: 171.92
MATIC: Short ma: 1.35, Long ma: 1.35
Available balance: 0.00286036 BTC
BTC: Short ma: 47376.24, Long ma: 47365.94
Available balance: 0.04012446 ETH
ETH: Short ma: 3388.00, Long ma: 3386.21
Available balance: 484.537 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
trade de venta pa: {'info': {'symbol': 'DOGEUSDT', 'orderId': '1765009047', 'orderListId': '-1', 'clientOrderId': 'x-R4BD3S82ff6c4e1a21c7f448e0ab4', 'transactTime': '1630431502316', 'price': '0.00000000', 'origQty': '484.00000000', 'executedQty': '484.00000000', 'cummulativeQuoteQty': '134.01960000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'fills': [{'price': '0.27690000', 'qty': '484.00000000', 'commission': '0.13401960', 'commissionAsset': 'USDT', 'tradeId': '325349807'}]}, 'id': '1765009047', 'clientOrderId': 'x-R4BD3S82ff6c4e1a21c7f448e0ab4', 'timestamp': 1630431502316, 'datetime': '2021-08-31T17:38:22.316Z', 'lastTradeTimes

DOGE: Short ma: 0.28, Long ma: 0.28
Available balance: 0.789118 LTC
LTC: Short ma: 172.23, Long ma: 172.02
MATIC: Short ma: 1.34, Long ma: 1.34
BTC: Short ma: 47288.49, Long ma: 47343.47
ETH: Short ma: 3378.27, Long ma: 3384.67
DOGE: Short ma: 0.28, Long ma: 0.28
Available balance: 0.789118 LTC
LTC: Short ma: 172.20, Long ma: 172.03
MATIC: Short ma: 1.34, Long ma: 1.34
BTC: Short ma: 47284.38, Long ma: 47342.97
ETH: Short ma: 3377.77, Long ma: 3384.62
DOGE: Short ma: 0.28, Long ma: 0.28
Available balance: 0.789118 LTC
LTC: Short ma: 172.18, Long ma: 172.04
MATIC: Short ma: 1.34, Long ma: 1.34
BTC: Short ma: 47279.51, Long ma: 47341.93
ETH: Short ma: 3377.30, Long ma: 3384.60
DOGE: Short ma: 0.28, Long ma: 0.28
Available balance: 0.789118 LTC
LTC: Short ma: 172.17, Long ma: 172.06
MATIC: Short ma: 1.34, Long ma: 1.34
BTC: Short ma: 47274.89, Long ma: 47340.54
ETH: Short ma: 3376.80, Long ma: 3384.53
DOGE: Short ma: 0.28, Long ma: 0.28
Available balance: 0.789118 LTC
LTC: Short ma: 172.1

LTC: Short ma: 171.88, Long ma: 172.25
MATIC: Short ma: 1.33, Long ma: 1.34
BTC: Short ma: 47164.26, Long ma: 47281.81
ETH: Short ma: 3369.36, Long ma: 3379.96
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 171.86, Long ma: 172.25
MATIC: Short ma: 1.33, Long ma: 1.34
BTC: Short ma: 47161.25, Long ma: 47280.20
ETH: Short ma: 3369.23, Long ma: 3379.87
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 171.84, Long ma: 172.25
MATIC: Short ma: 1.33, Long ma: 1.34
BTC: Short ma: 47159.03, Long ma: 47279.16
ETH: Short ma: 3369.20, Long ma: 3379.87
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 171.82, Long ma: 172.25
MATIC: Short ma: 1.33, Long ma: 1.34
BTC: Short ma: 47156.31, Long ma: 47277.86
ETH: Short ma: 3369.11, Long ma: 3379.85
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 171.80, Long ma: 172.26
MATIC: Short ma: 1.33, Long ma: 1.34
BTC: Short ma: 47153.77, Long ma: 47276.33
ETH: Short ma: 3369.05, Long ma: 3379.82
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 171.7

...

Available balance: 0.780337 LTC
LTC: Short ma: 173.26, Long ma: 172.79
Available balance: 100.2532 MATIC
MATIC: Short ma: 1.35, Long ma: 1.34
Available balance: 0.0028575 BTC
BTC: Short ma: 47304.81, Long ma: 47242.56
Available balance: 0.03968476 ETH
ETH: Short ma: 3410.13, Long ma: 3394.83
Available balance: 487.05 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
Available balance: 0.780337 LTC
LTC: Short ma: 173.27, Long ma: 172.81
Available balance: 100.2532 MATIC
MATIC: Short ma: 1.35, Long ma: 1.34
Available balance: 0.0028575 BTC
BTC: Short ma: 47306.48, Long ma: 47244.34
Available balance: 0.03968476 ETH
ETH: Short ma: 3410.47, Long ma: 3395.29
Available balance: 487.05 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
Available balance: 0.780337 LTC
LTC: Short ma: 173.26, Long ma: 172.82
Available balance: 100.2532 MATIC
MATIC: Short ma: 1.35, Long ma: 1.34
Available balance: 0.0028575 BTC
BTC: Short ma: 47305.16, Long ma: 47245.14
Available balance: 0.03968476 ETH
ETH: Short ma: 3410.46, Long

Available balance: 100.2532 MATIC
MATIC: Short ma: 1.35, Long ma: 1.34
Available balance: 0.0028575 BTC
BTC: Short ma: 47283.74, Long ma: 47258.38
Available balance: 0.03968476 ETH
ETH: Short ma: 3408.22, Long ma: 3400.69
Available balance: 487.05 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 172.96, Long ma: 172.99
Available balance: 100.2532 MATIC
MATIC: Short ma: 1.35, Long ma: 1.34
Available balance: 0.0028575 BTC
BTC: Short ma: 47283.03, Long ma: 47259.47
Available balance: 0.03968476 ETH
ETH: Short ma: 3408.26, Long ma: 3401.05
Available balance: 487.05 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 172.95, Long ma: 173.00
Available balance: 100.2532 MATIC
MATIC: Short ma: 1.35, Long ma: 1.34
Available balance: 0.0028575 BTC
BTC: Short ma: 47281.40, Long ma: 47260.30
Available balance: 0.03968476 ETH
ETH: Short ma: 3408.12, Long ma: 3401.32
Available balance: 487.05 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 172.94, Long ma: 173.00
Available balanc

Available balance: 100.2532 MATIC
MATIC: Short ma: 1.35, Long ma: 1.35
BTC: Short ma: 47246.33, Long ma: 47272.52
Available balance: 0.03968476 ETH
ETH: Short ma: 3408.86, Long ma: 3407.74
Available balance: 487.05 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 172.78, Long ma: 173.13
Available balance: 100.2532 MATIC
MATIC: Short ma: 1.35, Long ma: 1.35
BTC: Short ma: 47245.72, Long ma: 47273.18
Available balance: 0.03968476 ETH
ETH: Short ma: 3409.09, Long ma: 3408.10
Available balance: 487.05 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 172.79, Long ma: 173.14
Available balance: 100.2532 MATIC
MATIC: Short ma: 1.35, Long ma: 1.35
BTC: Short ma: 47244.57, Long ma: 47273.26
Available balance: 0.03968476 ETH
ETH: Short ma: 3409.29, Long ma: 3408.43
Available balance: 487.05 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 172.80, Long ma: 173.14
Available balance: 100.2532 MATIC
MATIC: Short ma: 1.35, Long ma: 1.35
BTC: Short ma: 47243.65, Long ma: 47272.62
A

BTC: Short ma: 47212.43, Long ma: 47256.62
ETH: Short ma: 3408.64, Long ma: 3409.03
Available balance: 487.05 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 172.72, Long ma: 173.03
MATIC: Short ma: 1.34, Long ma: 1.34
BTC: Short ma: 47204.53, Long ma: 47252.61
ETH: Short ma: 3408.01, Long ma: 3408.75
Available balance: 487.05 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 172.70, Long ma: 173.01
MATIC: Short ma: 1.34, Long ma: 1.34
BTC: Short ma: 47199.59, Long ma: 47250.25
ETH: Short ma: 3407.74, Long ma: 3408.70
Available balance: 487.05 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 172.67, Long ma: 172.99
MATIC: Short ma: 1.34, Long ma: 1.34
BTC: Short ma: 47193.89, Long ma: 47247.53
ETH: Short ma: 3407.36, Long ma: 3408.56
Available balance: 487.05 DOGE
DOGE: Short ma: 0.28, Long ma: 0.28
LTC: Short ma: 172.65, Long ma: 172.98
MATIC: Short ma: 1.34, Long ma: 1.34
BTC: Short ma: 47190.37, Long ma: 47245.74
ETH: Short ma: 3407.25, Long ma: 3408.54
Availabl

KeyboardInterrupt: 